In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [29]:
import os

In [156]:
# REGI 데이터 가져오기
datas = os.listdir('data/REGI')
li = []
for d in datas:
    df = pd.read_csv(f"data/REGI/{d}")
    li.append(df)

In [157]:
# 합병, 저장
regi = pd.concat(li)
regi.to_csv('./prepro_data/REGI_data.csv')

In [243]:
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions

ImportError: cannot import name 'Stylesheet' from 'bs4.element' (C:\Users\bitcamp\anaconda3\lib\site-packages\bs4\element.py)

In [34]:
# 재무제표 데이터 크롤링
url = 'https://www.choicestock.co.kr/search/invest/TSLA/MRQ'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
datas = soup.select('table.tableRanking')

In [ ]:
# 데이터 프레임 생성
li = []
for data in datas:
    table = parser_functions.make2d(data)
    df = pd.DataFrame(data=table[1:], columns=table[0])
    li.append(df)

In [ ]:
# 합병
df = pd.concat(li)

In [ ]:
# 저장
df.to_csv('prepro_data/fn_statement.csv')

In [163]:
# Date 전처리 함수
def change_date(x):
    ret = []
    for da in x.split():
        ret.append(da[:-1])
    return '.'.join(ret)

In [183]:
# Volumn 전치리 함수
def m_to_k(x):
    if x[-1] == 'M':
        return float(x[:-1]) * 1000
    elif x[-1] == 'B':
        return float(x[:-1]) * 1000000
    elif x[-1] == 'K':
        return float(x[:-1])
    else:
        return np.nan

In [244]:
# 알루미늄 전처리
df1 = pd.read_csv('data/알루미늄 내역.csv')
df1.drop(['오픈', '고가', '저가'], axis=1, inplace=True)
df1.columns = ['date', 'alu_price', 'alu_vol', 'alu_change']

In [245]:
df1.date = df1.date.apply(change_date)

In [246]:
df1.date = pd.to_datetime(df1.date)

In [247]:
df1['alu_price'] = df1['alu_price'].apply(lambda x: x.replace(',', ''))

In [248]:
df1['alu_price'] = pd.to_numeric(df1['alu_price'])

In [249]:
df1['alu_vol'] = df1['alu_vol'].apply(m_to_k)

In [250]:
df1['alu_change'] = df1['alu_change'].apply(lambda x: float(x.replace('%', '')))

In [251]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1815 non-null   datetime64[ns]
 1   alu_price   1815 non-null   float64       
 2   alu_vol     1814 non-null   float64       
 3   alu_change  1815 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 56.8 KB


In [252]:
# 구리 데이터 전처리
df2 = pd.read_csv('data/영국 구리 선물 내역.csv')
df2.drop(['오픈', '고가', '저가', '거래량'], axis=1, inplace=True)
df2.columns = ['date', 'cu_price', 'cu_change']

In [256]:
df2.date = df2.date.apply(change_date)

In [257]:
df2.date = pd.to_datetime(df2.date)

In [258]:
df2.cu_price = df2.cu_price.apply(lambda x: float(x.replace(',', '')))

In [259]:
df2['cu_change'] = df2['cu_change'].apply(lambda x: float(x.replace('%', '')))

In [181]:
# 알루미늄, 구리 데이터 합병
df_mg = pd.merge(df1, df2, on='date', how='outer')

In [194]:
# REGI 데이터 전처리
df3 = pd.read_csv('prepro_data/REGI_data.csv')
df3.drop(['Unnamed: 0', '오픈', '고가', '저가'], axis=1, inplace=True)
df3.columns = ['date', 'REGI_price', 'REGI_vol', 'REGI_change']

In [200]:
df3.date = df3.date.apply(change_date)

In [201]:
df3.date = pd.to_datetime(df3.date)

In [204]:
df3.REGI_vol = df3.REGI_vol.apply(m_to_k)

In [205]:
df3['REGI_change'] = df3['REGI_change'].apply(lambda x: float(x.replace('%', '')))

In [208]:
# REGI 합병
df_mg1 = pd.merge(df_mg, df3, on='date', how='outer')

In [211]:
# 합병한 데이터프레임 저장
df_mg1.to_csv('./prepro_data/alu_cu_REGI_df.csv')

In [ ]:
# 리튬 전처리 및 저장

In [222]:
df = pd.read_excel('data/리튬+가격2022-03-08.xls')

In [224]:
df.drop(['등락가', '등락비율', '재고량'], axis=1, inplace=True)

In [226]:
df.columns = ['date', 'li_price']

In [231]:
df.date = pd.to_datetime(df.date)

In [234]:
df.to_csv('prepro_data/data_li.csv')

In [400]:
df = pd.read_csv('prepro_data/테슬라 총합.csv')

In [414]:
# y = df.tesla
# x = df.drop(['Unnamed: 0', 'date', 'tesla'], axis=1)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lg = LogisticRegression()
# rid = Ridge()
# las = Lasso()

models = [lr, rid, las]

for model in models:
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    print(model.__class__.__name__)
    print('r2_score :', r2_score(y_test, preds))
    print('rmse :', np.sqrt(mean_squared_error(y_test, preds)))

LinearRegression
r2_score : 0.15786761258417115
rmse : 0.4563038358649629
Ridge
r2_score : 0.1579368576315302
rmse : 0.4562850754950131
Lasso
r2_score : 0.024409340687458037
rmse : 0.49113145670303243


In [415]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)

In [512]:
df_total = pd.read_csv('prepro_data/테슬라 총합.csv')

In [267]:
y = df.tesla
x = df.drop(['Unnamed: 0', 'date', 'tesla'], axis=1)

In [268]:
from sklearn.model_selection import train_test_split

In [275]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)

In [300]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

In [303]:
lr = LinearRegression()
rid = Ridge()
las = Lasso

models = [lr, rid, las]

In [304]:
for model in models:
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    print(model.__class__.__name__)
    print('r2_score :', r2_score(y_test, preds))

LinearRegression
r2_score : 0.9802214926606811
Ridge
r2_score : 0.9802204979242496


TypeError: fit() missing 1 required positional argument: 'y'

In [290]:
lr.fit(x_train, y_train)

LinearRegression()

In [291]:
preds = lr.predict(x_test)

In [297]:
r2_score(y_test, preds)

0.9802214926606811

In [299]:
np.sqrt(mean_squared_error(y_test, preds))

41.816757742474046

In [ ]:
lg = LogisticRegression()

In [513]:
df_cha = pd.read_csv('data/TSLA 역사적 데이터.csv')

In [514]:
df_cha['change'] = df_cha['변동 %'].apply(lambda x: 0 if float(x[:-1]) < 0 else 1)

In [515]:
df_cha = df_cha[['날짜', 'change']].rename(columns = {'날짜': 'date'})

In [516]:
df_cha.date = pd.to_datetime(df_cha.date.apply(change_date))

In [517]:
df_cha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1808 entries, 0 to 1807
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1808 non-null   datetime64[ns]
 1   change  1808 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 28.4 KB


In [518]:
df_total.date = pd.to_datetime(df_total.date)

In [566]:
df_cla = pd.merge(df_total, df_cha, on='date', how='left')

In [567]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [568]:
df_cla.date

0      2015-01-02
1      2015-01-05
2      2015-01-06
3      2015-01-07
4      2015-01-08
          ...    
1802   2022-02-28
1803   2022-03-01
1804   2022-03-02
1805   2022-03-03
1806   2022-03-04
Name: date, Length: 1807, dtype: datetime64[ns]

In [569]:
le.fit(df_cla.date)

LabelEncoder()

In [570]:
df_cla.date = le.transform(df_cla.date)

In [571]:
df_cla

,Unnamed: 0,date,tesla,tesla_거래량,tesla_volatility,elec_fee,oli_price,oli_price_volatility,carbon_credits,carbon_credits_volatility,...,dollar_volatility,alu_price,alu_vol,alu_change,cu_price,cu_change,REGI_price,REGI_vol,REGI_change,change
0,0,0,43.86,23820.0,-1.39,6.67,52.69,-3.44,7.01,-3.31,...,0.90,1828.00,31.38,-1.12,6250.25,-0.55,9.79,436.60,0.82,0
1,1,1,42.02,26840.0,-4.20,6.67,50.04,-5.03,6.90,-1.57,...,0.33,1812.25,43.25,-0.86,6142.00,-1.73,9.55,481.67,-2.45,0
2,2,2,42.26,31310.0,0.57,6.67,47.93,-4.22,6.77,-1.88,...,0.13,1780.25,56.01,-1.77,6131.00,-0.18,9.12,534.76,-4.50,1
3,3,3,42.19,14840.0,-0.17,6.67,48.65,1.50,6.78,0.15,...,0.43,1783.50,61.45,0.18,6125.00,-0.10,9.19,267.66,0.77,0
4,4,4,42.12,17210.0,-0.17,6.67,48.79,0.29,6.81,0.44,...,0.52,1825.50,75.03,2.35,6122.75,-0.04,9.43,359.25,2.61,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,1802,1802,870.43,32770.0,7.48,7.16,95.72,4.51,82.65,-5.83,...,0.10,3368.50,85.69,0.33,9883.50,0.11,61.50,12740.00,40.38,1
1803,1803,1803,864.37,24720.0,-0.70,7.16,103.41,8.03,69.74,-15.62,...,0.73,3478.00,68.56,3.25,10061.50,1.80,60.75,4790.00,-1.22,0
1804,1804,1804,879.89,24650.0,1.80,7.16,110.60,6.95,67.72,-2.90,...,-0.02,3569.00,68.40,2.62,10166.00,1.04,60.86,5040.00,0.18,1
1805,1805,1805,839.29,20260.0,-4.61,7.16,107.67,-2.65,67.01,-1.05,...,0.41,3716.50,106.23,4.13,10351.00,1.82,61.26,5430.00,0.66,0


In [572]:
y = df_cla.change
x = df_cla.drop(['Unnamed: 0', 'change'], axis=1)

In [579]:
y_train = y[:1444]
y_test = y[1444:]
x_train = x[:1444]
x_test = x[1444:]

In [582]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [583]:
lg = LogisticRegression()
lg.fit(x_train, y_train)

LogisticRegression()

In [584]:
preds = lg.predict(x_test)

In [585]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.46      0.99      0.63       167
           1       0.50      0.01      0.01       196

    accuracy                           0.46       363
   macro avg       0.48      0.50      0.32       363
weighted avg       0.48      0.46      0.29       363



In [586]:
from sklearn.preprocessing import StandardScaler

In [601]:
scaler = StandardScaler()

In [602]:
scaler.fit(x_train)

StandardScaler()

In [603]:
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [604]:
lg.fit(x_train_scaled, y_train)

LogisticRegression()

In [605]:
preds = lg.predict(x_test_scaled)

In [606]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      0.40      0.57       167
           1       0.66      1.00      0.80       196

    accuracy                           0.72       363
   macro avg       0.83      0.70      0.68       363
weighted avg       0.82      0.72      0.69       363



In [607]:
from sklearn.ensemble import RandomForestClassifier

In [625]:
rfc = RandomForestClassifier(n_estimators=5)

In [626]:
rfc.fit(x_train, y_train)

RandomForestClassifier(n_estimators=5)

In [627]:
preds = rfc.predict(x_test)

In [628]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.81      0.96      0.88       167
           1       0.96      0.81      0.88       196

    accuracy                           0.88       363
   macro avg       0.89      0.89      0.88       363
weighted avg       0.89      0.88      0.88       363



In [629]:
y_train

0       0
1       0
2       1
3       0
4       0
       ..
1439    1
1440    1
1441    0
1442    0
1443    1
Name: change, Length: 1444, dtype: int64

5